In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
import torch
from sbi.inference import prepare_for_sbi, SNLE
from sbi.simulators.linear_gaussian import diagonal_linear_gaussian
import sbi.utils as sbi_utils

### Load theta and x

In [28]:
data = np.load('../../../arco_sims/cleaned_up_data.npz', allow_pickle=True)

x = torch.as_tensor(data['data'], dtype=torch.float32)
theta = torch.as_tensor(data['params'], dtype=torch.float32)

### Load prior min and max and gt

In [29]:
import sys
sys.path.append('../')

In [30]:
from parameter_setup import load_ground_truth_params, load_prior_min, load_prior_max

### Load data to standardize - needed to standardize x_o

In [31]:
#standardize_vals = np.load('../results/standardize_vals.npz')
#x_mean = standardize_vals['data_mean']
#x_std = standardize_vals['data_std']
#
#theta_mean = standardize_vals['theta_mean']
#theta_std = standardize_vals['theta_std']

x_mean = x.mean(dim=0)
x_std = x.std(dim=0)

theta_mean = theta.mean(dim=0)
theta_std = theta.std(dim=0)

In [32]:
x = (x-x_mean) / x_std
theta = (theta-theta_mean) / theta_std

### Load x_o (which I got from running the simulator with Arco's ground truth params)

In [33]:
import numpy as np
x_o = torch.as_tensor(np.load('../results/observation/x_o.npz')['x_o'], dtype=torch.float32)

x_o = (x_o - x_mean) / x_std
x_o = x_o.unsqueeze(0)

### Create dummy simulator and dummy prior

In [37]:
def dummy_simulator(theta):
    return torch.ones(1,35)

dummy_prior = sbi_utils.BoxUniform(torch.as_tensor(load_prior_min()), torch.as_tensor(load_prior_max()))
_bound = torch.sqrt(torch.as_tensor(3.))
dummy_prior_norm = sbi_utils.BoxUniform(-_bound*torch.ones(35), _bound*torch.ones(35))

In [38]:
simulator, prior, x_shape = prepare_for_sbi(dummy_simulator, dummy_prior_norm)

inference = SNLE(
    simulator, 
    prior,
    x_shape,
    external_data=(theta, x),
    mcmc_method='slice', 
    density_estimator='nsf'
)

In [39]:
posterior = inference(
    num_rounds=1,
    num_simulations_per_round=0,
    batch_size=100 # default is 50
    stop_after_epochs=3, # default is 20
)

KeyboardInterrupt: 

### Analyse posterior

In [16]:
posterior = posterior.set_default_x(x=x_o)

In [17]:
posterior._mcmc_method = 'slice'

In [19]:
samples = posterior.sample((10000,), num_chains=20, thin=1, show_progress_bars=True)

# denormalize the samples
samples = samples * theta_std + theta_mean

# convert to list for pickling
samples_list = samples.numpy().tolist()

In [29]:
import pickle
with open('../results/posteriors/200616_dummyPosteriorSNL.pickle', 'wb') as handle:
    pickle.dump(posterior, handle)

In [36]:
with open('../results/samples/200616_dummyPosteriorSNL_samples.pickle', 'wb') as handle:
    pickle.dump(samples_list, handle, protocol=2)

### Posterior predictives
Has to be done in a different virtual env with python 2.